In [ ]:
import os

%cd /content/drive/MyDrive/NLP
print(os.getcwd())

/content/drive/MyDrive/NLP
/content/drive/MyDrive/NLP


In [ ]:
!pip install torch --quiet
!pip install sentence-transformers --quiet
!pip install scikit-learn --quiet
!pip install transformers --quiet
!pip install faiss-gpu --quiet
!pip install langchain --quiet
!pip install langchain-community --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.9 MB/s eta 0:00:00


In [ ]:
import json
import torch
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from transformers import pipeline
from langchain.vectorstores import FAISS
from langchain.embeddings import SentenceTransformerEmbeddings  # Or appropriate embeddings class
import os

In [ ]:
# Define file paths
train_path = 'train_TLQA.json'
val_path = 'val_TLQA.json'
test_path = 'test_TLQA.json'

# Load JSON data
with open(train_path, 'r') as f_train, open(val_path, 'r') as f_val, open(test_path, 'r') as f_test:
    train_data = json.load(f_train)
    val_data = json.load(f_val)
    test_data = json.load(f_test)

In [ ]:
embedding_model_name = 'sentence-transformers/all-mpnet-base-v2'
embedding_model = SentenceTransformer(embedding_model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
class KnnSearch:
    def __init__(self, model_name='sentence-transformers/all-mpnet-base-v2'):
        self.model = SentenceTransformer(model_name)

    def get_embeddings_for_data(self, data_ls):
        embeddings = self.model.encode(data_ls, convert_to_tensor=True, show_progress_bar=True)
        return embeddings.cpu().numpy()

    def get_top_n_neighbours(self, sentence, data_emb, transfer_data, k):
        sent_emb = self.model.encode([sentence], convert_to_tensor=True)
        sent_emb = sent_emb.cpu().numpy()
        text_sims = cosine_similarity(data_emb, sent_emb).flatten()
        top_indices = text_sims.argsort()[-k:][::-1]
        top_questions = [transfer_data[idx] for idx in top_indices]
        return top_questions

knnSearch = KnnSearch(model_name=embedding_model_name)

In [ ]:
n_to_load = 20

train_questions = [item["question"] for item in train_data]
train_questions_embedded = knnSearch.get_embeddings_for_data(train_questions)

test_questions = [item["question"] for item in test_data[:n_to_load]]
test_answers = [item["answers"] for item in test_data[:n_to_load]]

Batches:   0%|          | 0/101 [00:00<?, ?it/s]

In [ ]:
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device Name: {torch.cuda.get_device_name(0)}")

CUDA Available: True
Device Name: Tesla T4


In [ ]:
# Ensure you have the 'google/flan-t5-large' model downloaded or accessible
pl = pipeline("text2text-generation", model="google/flan-t5-large", device=0 if torch.cuda.is_available() else -1)

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
# Define the path to your FAISS vector store
faiss_store_path = 'faiss_db_final'

# Ensure the path exists
if not os.path.exists(faiss_store_path):
    raise FileNotFoundError(f"FAISS vector store not found at {faiss_store_path}")

# Initialize embeddings for FAISS (ensure consistency with how FAISS was created)
faiss_embeddings = SentenceTransformerEmbeddings(model_name="sentence-transformers/msmarco-distilbert-base-tas-b")

# Load the FAISS vector store using LangChain
faiss_store = FAISS.load_local(faiss_store_path, embeddings=faiss_embeddings, allow_dangerous_deserialization=True)

<ipython-input-10-995da8cf4590>:9: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  faiss_embeddings = SentenceTransformerEmbeddings(model_name="sentence-transformers/msmarco-distilbert-base-tas-b")


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
def generate_prompt(few_shot_examples, contexts, question):
    prompt = f'''You are a Timeline based Question Answering language model.
    Below are {len(few_shot_examples)} examples of questions and answers, showing the structure you should use for your answers.
    After the examples a context is provided. This context contains the information that is necessary to answer the final question.
    The final question is given at the end of this prompt.
    '''

    # Add Few-Shot Examples
    for few_shot_example in few_shot_examples:
        prompt += f"Question: {few_shot_example['question']}\n"
        prompt += f"Answer: {'; '.join(few_shot_example['answers'])}\n\n"

    # Add Retrieved Contexts
    if contexts:
        prompt += "Context:\n"
        for ctx in contexts:
            prompt += f"{ctx.page_content}\n"
        prompt += "\n"



    # Add the Test Question
    prompt += f"Question: {question}\nAnswer: "
    return prompt

In [ ]:
def generate_json(questions, generated_answers, ground_truths):
    data = []
    for question, generated, ground_truth in zip(questions, generated_answers, ground_truths):
        entry = {
            "question": question,
            "generated_answer": generated.split("; "),
            "ground_truth": ground_truth
        }
        data.append(entry)

    # Write the list of dictionaries to a JSON file
    with open("output_large_k_5_with_context_no_few_shot.json", "w") as json_file:
        json.dump(data, json_file, indent=4)

    print("JSON file with context created successfully!")

In [ ]:
def test_run(questions, ground_truths, k_few_shot=10, k_context=5):
    all_prompts = []
    for idx, question in enumerate(questions):
        # Retrieve Top-k Few-Shot Examples using KNN Search
        most_similar_questions = knnSearch.get_top_n_neighbours(
            sentence=question,
            data_emb=train_questions_embedded,
            transfer_data=train_data,
            k=k_few_shot
        )

        # Retrieve Top-k Contexts using FAISS
        if k_context > 0:
          retrieved_contexts = faiss_store.similarity_search(query=question, k=k_context)
        # retrieved_contexts is a list of Document objects; adjust attribute access as needed
        else:
          retrieved_contexts = []

        # Generate Prompt with Context and Few-Shot Examples
        prompt = generate_prompt(
            few_shot_examples=most_similar_questions,
            contexts=retrieved_contexts,
            question=question
        )
        all_prompts.append(prompt)

        if (idx + 1) % 100 == 0:
            print(f"Prepared prompts for {idx + 1} questions")

    print("All prompts prepared. Starting generation...")

    # Generate Answers in Batches
    outputs = []
    batch_size = 5
    for i in range(0, len(all_prompts), batch_size):
        batch_prompts = all_prompts[i:i + batch_size]
        batch_outputs = pl(batch_prompts, max_new_tokens=100)  # Increased token limit for context
        outputs.extend(batch_outputs)
        print(f"Processed batch {i // batch_size + 1}")

    # Extract Generated Answers
    generated_answers = [entry['generated_text'].strip() for entry in outputs]

    # Generate JSON Output
    generate_json(questions, generated_answers, ground_truths)

In [ ]:
test_run(test_questions, test_answers, k_few_shot=1, k_context=1)

All prompts prepared. Starting generation...
Processed batch 1
Processed batch 2
Processed batch 3
Processed batch 4
JSON file with context created successfully!
